In [ ]:
# This notebook is used for testing learning using the draculab module.

# This notebook is a copy of test3.ipynb, with two mutually inhibiting units instead of one unit.

# By Sergio Verduzco Flores        June 2017

In [ ]:
# This file is supposed to be in .../draculab/tests/ , so cd before importing:
%cd ..
from draculab import *
import numpy as np
import time
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

In [ ]:
def conn_mat(net):
    # Get the connection matrix of a network in a Numpy array
    conns = np.zeros((net.n_units,net.n_units))
    for syn_list in net.syns:
        for syn in syn_list:
            conns[syn.postID,syn.preID] = syn.w
    return conns

def plot_stuff(data):
    #fig.clf()
    #plt.close()
    fig = plt.figure(figsize=(10,6))
    for i in range(9):
        ax = fig.add_subplot(3,3,i+1)
        ax.clear()
        plt.ylim(-0.01,0.5)
        ax.plot(data[0], data[1][i], 'r', figure=fig)
    new_fig = plt.figure(figsize=(5,3))
    plt.plot(sim_dat[0], sim_dat[1][9], 'b', sim_dat[0], sim_dat[1][10], 'k', figure=new_fig)
    fig.canvas.draw()
    plt.show()
    

In [ ]:
################# TEST 1 ####################
######### 1) Create a network
net_params = {'min_delay' : 0.1, 'min_buff_size' : 10 } # parameter dictionary for the network
n1 = network(net_params)

######### 2) Put some units in the network
# default parameters for the units
pars = { 'coordinates' : np.array([0., 0., 0.]), 'function' : lambda x:None,
         'init_val' : 0.5, 'tau_fast' : .2, 'tau_slow' : 4.,
         'slope' : 2., 'thresh' : 0.2, 'tau' : 0.05,
         'type' : unit_types.source } 
inputs = n1.create(9,pars) # creating nine input sources
pars['type'] = unit_types.sigmoidal
units = n1.create(2,pars) # creating two sigmoidal units

######### 3) Connect the units in the network
conn_spec = {'rule' : 'all_to_all', 'delay' : [1.]*9 + [1.1]*9,
             'allow_autapses' : False} # connection specification dictionary
syn_pars = {'init_w' : {'distribution':'uniform', 'low':0.1, 'high':0.5}, 
            'lrate' : 0.02, 'type' : synapse_types.bcm} # synapse parameters dictionary
n1.connect(inputs, units, conn_spec, syn_pars)

# We put mutual inhibition in the units to see if they can learn to extract different features
syn_pars['init_w'] = [-2., -2.]
syn_pars['type'] = synapse_types.static
conn_spec['delay'] = [0.9, 1.1]
n1.connect(units, units, conn_spec, syn_pars)

######### 4) Running and visualizing 

####### SETTING THE INPUT FUNCTIONS
### You are going to present 4 input patterns that randomly switch over time.
### Imagine the 9 inputs arranged in a grid, like a tic-tac-toe board, numbered
### from left to right and from top to bottom:
### 1 2 3
### 4 5 6
### 7 8 9
### You'll have input patterns
### 0 X 0   0 0 0   X 0 X   0 X 0
### 0 X 0   X X X   0 0 0   X 0 X
### 0 X 0   0 0 0   X 0 X   0 X 0
### The input is always a normalized linear combination of one or two of these patterns.
### Pattern pat1 is presented alone for t_pat time units, and then there is a transition period
### during which pat1 becomes pat2 by presenting at time t an input 
### c*(t_pat+t_trans - t)*pat1 + c*(t - tpat)*pat2
### where c = 1/t_trans, and t_trans is the duration of the transition period. 
### At time t_pat+t_trans, pat2 is presented by itself for t_pat time units.
### 
# here are the patterns as arrays
patterns = [np.zeros(9) for i in range(4)]
patterns[0] = np.array([0., 1., 0., 0., 1., 0., 0., 1., 0.])/3.
patterns[1] = np.array([0., 0., 0., 1., 1., 1., 0., 0., 0.])/3.
patterns[2] = np.array([1., 0., 1., 0., 0., 0., 1., 0., 1.])/4.
patterns[3] = np.array([0., 1., 0., 1., 0., 1., 0., 1., 0.])/4.

n_pres = 100
# number of times some pattern will be presented
t_pat = 10. # as above
t_trans = 4.
c = 1/t_trans # auxiliary variable
pats = range(4) # auxiliary variable
cur_pat = np.random.choice(pats)  # pattern currently presented
next_pat = np.random.choice(pats) # next pattern to be presented
last_t = 0.
start_time = time.time()

def make_fun1(idx):  
    # This creates a constant function with value: patterns[cur_pat][idx]
    # thus avoiding a scoping problem that is sometimes hard to see:
    # https://eev.ee/blog/2011/04/24/gotcha-python-scoping-closures/
    fun = lambda t : patterns[cur_pat][idx]
    return fun

def make_fun2(idx, last_t):
    # Creates a function for the pattern transition
    fun = lambda t : c * ( (t_trans - (t-last_t))*patterns[cur_pat][idx] +
                           (t-last_t)*patterns[next_pat][idx] )
    return fun

for pres in range(n_pres):
# For each cycle you'll set the input functions and simulate, once with a single pattern,
# once with a mix of patterns, as described above
    
    # first, we present a single pattern
    for u in range(9):
        n1.units[inputs[u]].set_function( make_fun1(u) )
        
    sim_dat = n1.flat_run(t_pat)  # simulating
    last_t = n1.sim_time # simulation time after last pattern presentation
    
    #plot_stuff(sim_dat,fig)
    #input('Fixed pattern presented: ' + str(cur_pat))
    
    # now one pattern turns into the next
    for u in range(9):
        n1.units[inputs[u]].set_function(make_fun2(u, last_t))
    
    sim_dat = n1.flat_run(t_trans) # simulating
    
    #plot_stuff(sim_dat,fig)
    #input('Transitioned to pattern: ' + str(next_pat))

    # choose the pattern you'll present next
    cur_pat = next_pat
    next_pat = np.random.choice(pats)
    
    if pres%10 == 0:
        print('Presentation ' + str(pres+1))

print('Execution time: %s seconds' % (time.time() - start_time))
# 72.14904570579529 seconds   -- initially
# 7.2 seconds -- typical

In [ ]:
# Checking whether the units have preferred patterns

n_pres = 4
cur_pat = 0
next_pat = 1
for pres in range(n_pres):
# For each cycle you'll set the input functions and simulate, once with a single pattern,
# once with a mix of patterns, as described above
    # first, we present a single pattern
    
    for u in range(9):
        n1.units[inputs[u]].set_function( make_fun1(u) )
        
    sim_dat = n1.flat_run(t_pat)  # simulating
    last_t = n1.sim_time # simulation time after last pattern presentation
    
    plot_stuff(sim_dat)
    #input('Transitioned to pattern: ' + str(next_pat))
    print('Transitioned to pattern: ' + str(next_pat))
    
    # now one pattern turns into the next
    for u in range(9):
        n1.units[inputs[u]].set_function(make_fun2(u, last_t))
    
    sim_dat = n1.flat_run(t_trans) # simulating
    
    plot_stuff(sim_dat)
    #input('Transitioned to pattern: ' + str(next_pat))
    print('Transitioned to pattern: ' + str(next_pat))

    # choose the pattern you'll present next
    cur_pat = next_pat
    next_pat = (next_pat+1)%4
    
    if pres%10 == 0:
        print('Presentation ' + str(pres+1))
        
weights1 = (np.array(n1.units[9].get_weights(n1.sim_time)))[:-1]  
weights2 = (np.array(n1.units[10].get_weights(n1.sim_time)))[:-1] 
proj1 = []
proj2 = []
for i in range(4):
    proj1.append(sum(weights1*patterns[i]))
    proj2.append(sum(weights2*patterns[i]))
print(proj1)
print(proj2)

In [ ]:
#### Comparing the weight vectors with the leading eigenvector of the correlation matrix
#######################################################################################

plt.ion()
weights1 = (np.array(n1.units[9].get_weights(n1.sim_time)))[:-1]  # final input weights of the linear unit 1
weights2 = (np.array(n1.units[10].get_weights(n1.sim_time)))[:-1] # final inputweights of the linear unit 2

# Obtaining eigenvectors of the correlation matrix
pat_mat = np.matrix(patterns)
corr = (pat_mat.T)*pat_mat # input correlation matrix
eigs = np.linalg.eig(corr) # extracting eigenvalues and eigenvectors
evals = eigs[0] # eigenvalues
evecs = [eigs[1][:,i] for i in range(9)] # eigenvectors

# obtaining the leading eigenvector
max_index, max_value = max(enumerate(evals), key=lambda p:p[1])
print('Max eigenvalue: ' + str(max_value) + ', index: ' + str(max_index))
max_evector = evecs[max_index]
    
# plotting all eigenvectors in 3x3 format
fig3 = plt.figure(figsize=(10,10))
ev_grid = ImageGrid(fig3, 111, nrows_ncols=(3,3), axes_pad=0.05)
for idx,vec in enumerate(evecs):
    vec.shape = 3,3
    ev_grid[idx].imshow(vec)

# plotting leading eigenvector VS weight vectors in 3x3 format
fig4 = plt.figure(figsize=(18,5))
sp_ev_grid = ImageGrid(fig4, 111, nrows_ncols=(1,3), axes_pad=0.2)
for idx,vec in enumerate([-max_evector, weights1, weights2]):
    vec.shape = 3,3
    sp_ev_grid[idx].imshow(vec)
    print(["%.2f" % float(v[0]) for v in vec.reshape(9,1)])
